In [ ]:
import ROOT

In [ ]:
%jsroot on

In [ ]:
ROOT.gStyle.SetPalette(ROOT.kBird)
ROOT.gStyle.SetNumberContours(256)
ROOT.gStyle.SetMarkerSize(2)
#ROOT.gStyle.SetErrorX(0)
ROOT.gStyle.SetPaintTextFormat(".2f")
ROOT.gStyle.SetOptStat(0)
ROOT.gStyle.SetPadRightMargin(0.05)
#ROOT.gStyle.SetMarkerStyle(ROOT.kPlus)
ROOT.gStyle.SetMarkerStyle(ROOT.kDot)

In [ ]:
plotDir = "comp-plots"

In [ ]:
inputdir = "../outputs/"
#d = ROOT.RDataFrame("pfos", (f"{inputdir}/ZH_eLpR_500k_bb.root", f"{inputdir}/ZH_eRpL_500k_bb.root"))
d = ROOT.RDataFrame("pfos", f"{inputdir}/ZH_eLpR_test_bb.root")
print([n for n in d.GetColumnNames()])

In [ ]:
d = (d
     .Define("shape0_over_p", "shape0 / seenP")
     .Define("ecal_over_p", "seenEcalDep / seenP") # just looks like ecal_share
     .Define("ecalmp_over_ecalpp", "(seenEcalDep - seenP) / (seenEcalDep + seenP)")
    )

In [ ]:
used_vars = ["e_over_p", "ecal_share", "seenEcalDep", "seenYokeDep",            
             "cluEllipsoid_r1", "cluEllipsoid_r2", "cluEllipsoid_r3", "cluEllipsoid_r_ave", "cluEllipsoid_density",
             "cluEllipsoid_eccentricity_T", "cluEllipsoid_eccentricity_L",
             "dEdxDist_e"]
used_vars += [f"shape{i}" for i in range(24)]
#used_vars += ["shape8", "shape21", "shape19", "shape0", "shape1"]
used_vars += ["ecalmp_over_ecalpp"]

In [ ]:
v_map = {
    "shape0": "npoints",
    "shape1": "sum_wgt",
    "shape4": "chi2",
    "shape7": "absorption_Length",
    "shape8": "showerMax_electron",
    "shape10": "Rm",
    "shape17": "max_Ed_hit",
    "shape19": "xt90",
    "shape20": "xl20",
    "shape21": "depth_of_cluster",
    "shape22": "meanOf_hitradius",
    "shape23": "rmsOf_hitradius",
}

In [ ]:
# for v in used_vars:
#     d = d.Filter(f"std::isfinite({v})", f"test_{v}")

In [ ]:
# d.Report().Print()

In [ ]:
d_e = d.Filter("truePDG == 11")
d_mu = d.Filter("truePDG == 13")
d_had = d.Filter("truePDG == 211 || truePDG == 321 || truePDG == 2212")

In [ ]:
used_vars

In [ ]:
h_e = {}
h_mu = {}
h_had = {}

nbins = 10 #50
lbin = 0
hbin = 10 # 25

for v in used_vars:
    h_e[v] = d_e.Profile1D(("", f";seenP;{v}", nbins, lbin, hbin, "S"), "seenP", v)
    h_mu[v] = d_mu.Profile1D(("", f";seenP;{v}", nbins, lbin, hbin, "S"), "seenP", v)
    h_had[v] = d_had.Profile1D(("", f";seenP;{v}", nbins, lbin, hbin, "S"), "seenP", v)

In [ ]:
c = {}
s = {}
m = {}
p_e = {}
p_mu = {}
p_had = {}
l = {}
g_e = {}
g_mu = {}
g_had = {}
for var in used_vars:
    c[var] = ROOT.TCanvas()
    #s[var] = ROOT.THStack()
    m[var] = ROOT.TMultiGraph()
    p_e[var] = h_e[var].ProjectionX()
    p_e[var].SetFillColorAlpha(ROOT.kBlue, 0.33)
    p_e[var].SetLineColor(ROOT.kBlue)
    p_e[var].SetMarkerColor(ROOT.kBlue)
    #s[var].Add(p_e[var])
    g_e[var] = ROOT.TGraphErrors(p_e[var])
    m[var].Add(g_e[var])

    p_mu[var] = h_mu[var].ProjectionX()
    p_mu[var].SetFillColorAlpha(ROOT.kRed, 0.33)
    p_mu[var].SetLineColor(ROOT.kRed)
    p_mu[var].SetMarkerColor(ROOT.kRed)
    #s[var].Add(p_mu[var])
    g_mu[var] = ROOT.TGraphErrors(p_mu[var])
    m[var].Add(g_mu[var])

    p_had[var] = h_had[var].ProjectionX()
    p_had[var].SetFillColorAlpha(ROOT.kBlack, 0.33)
    p_had[var].SetLineColor(ROOT.kBlack)
    p_had[var].SetMarkerColor(ROOT.kBlack)
    #s[var].Add(p_had[var])
    g_had[var] = ROOT.TGraphErrors(p_had[var])
    m[var].Add(g_had[var])

    m[var].Draw("al3")
    
    #s[var].Draw("nostack e5")
   # s[var].Draw("nostack same lclusterEnergy")

    #s.SetTitle(";Momentum [GeV]; #frac{(dE/dx)_{meas.} - (dE/dx)_{e}}{#sigma_{dE/dx}}")
    #s[var].SetTitle(f";Momentum [GeV];{var}")
    v_name = v_map[var] if var in v_map else var
    m[var].SetTitle(f";Momentum [GeV];{v_name}")
    l[var] = ROOT.TLegend(0.5, 0.2, 0.8, 0.45)
    l[var].AddEntry(p_e[var], "electrons (stddev)")
    l[var].AddEntry(p_mu[var], "muons (stddev)")
    l[var].AddEntry(p_had[var], "hadrons (stddev)")
    l[var].Draw()
    c[var].Draw()
    c[var].SaveAs(f"{plotDir}/{var}.pdf")

#c.SaveAs("test.pdf")